In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import matplotlib
import time
import numpy as np
%matplotlib inline

import pandas as pd
pd.options.display.max_colwidth = 300
pd.options.display.max_columns = 100



In [6]:
#Model File Directory
WORK_DIR = '/drhdev/workspace/ml/'

#Label Keys
LABEL = "success"

MODEL_TYPE = "ML-BR"

start_date = '2018-01-01'
end_date = '2018-06-30'
site_ids = ['avast', 'kasperus']
# site_ids = ['avast', 'kasperus', 'adbehap', 'adbehbr', 'adbehkr', 'mcafeeus', 'mfeap', 'mfeeu']
desc = str('%s_%s_%s' % (start_date, end_date, "_".join(site_ids)))

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit
from pyspark.sql.functions import pandas_udf, PandasUDFType

In [10]:
def convert_status(value):
    if value == 'Completed' or value == 'Reversed':
        return 1
    else:
        return 0
    
    
def cast_to_int(value, default=-1):
    try:
        return int(value)
    except (ValueError, TypeError):
        return default


In [9]:
import datetime
from datetime import timedelta
import time

# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn import preprocessing
import calendar, datetime
import time
# from sklearn.preprocessing import LabelEncoder

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

#Determines the week (number) of the month
def week_of_month(datestr):
    date = to_date(datestr)
    #Calendar object. 6 = Start on Sunday, 0 = Start on Monday
    cal_object = calendar.Calendar(6)
    month_calendar_dates = cal_object.itermonthdates(date.year,date.month)

    day_of_week = 1
    week_number = 1

    for day in month_calendar_dates:
        #add a week and reset day of week
        if day_of_week > 7:
            week_number += 1
            day_of_week = 1

        if date == day:
            break
        else:
            day_of_week += 1

    return str(week_number)

def to_weekday(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date.isoweekday()

def daterange(date1_str, date2_str):
    start_date = to_date(date1_str + ' 00:00:00')
    end_date = to_date(date2_str + ' 00:00:00')
    if start_date > end_date:
        raise ValueError('start date cannot be after the end date.')
    for n in range(int((end_date - start_date).days) + 1):
        yield str(start_date + timedelta(n))
        
def date_range_condition(start_date, end_date):
    dates = daterange(start_date, end_date)
    for idx, date in enumerate(dates):
        date_formatted = date.replace('-','')
        if(idx == 0):
            yield str("received_date = '%s'" % date_formatted)
        else:
            yield str(" or received_date = '%s'" % date_formatted)
        

In [11]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

udfValueToStatus = udf(convert_status, StringType())  
date_condition = "".join(date_range_condition(start_date, end_date))
# site_date_condition = str("( (%s) and site_id = 'avast')" % date_condition)
site_date_condition = str("( %s )" % date_condition)
print(site_date_condition)
sdf = spark.read\
    .format("org.apache.spark.sql.cassandra")\
    .options(table="cpg_transaction", keyspace="subs")\
    .load()
    
sdf =  sdf.filter(site_date_condition)\
    .filter("!(renew_att_num == '1' and new_status in ('Completed','Reversed'))")\
    .repartition(8, "subsegment_id")\
    .withColumn("success", udfValueToStatus("new_status"))\
    .withColumn("day_of_month", sdf["day_of_month"].cast(IntegerType()))\
    .cache()
    
    #     .repartition(8, "subscription_id")\
#   .repartition("subscription_id")\
#     .filter("(received_date = '20180110' or received_date = '20180115') and site_id = 'avast'")\

grouped_schema = sdf.schema.add("first_attempt_date", StringType())\
                            .add("first_day_of_month", IntegerType())\
                            .add("first_transaction_hour", StringType())\
                            .add("first_response_code", StringType())\
                            .add("first_response_message", StringType())\
                            .add("first_response_group", StringType())\
                            .add("first_decline_type", StringType())\
                            .add("days_between", IntegerType()) 


( received_date = '20180101' or received_date = '20180102' or received_date = '20180103' or received_date = '20180104' or received_date = '20180105' or received_date = '20180106' or received_date = '20180107' or received_date = '20180108' or received_date = '20180109' or received_date = '20180110' or received_date = '20180111' or received_date = '20180112' or received_date = '20180113' or received_date = '20180114' or received_date = '20180115' or received_date = '20180116' or received_date = '20180117' or received_date = '20180118' or received_date = '20180119' or received_date = '20180120' or received_date = '20180121' or received_date = '20180122' or received_date = '20180123' or received_date = '20180124' or received_date = '20180125' or received_date = '20180126' or received_date = '20180127' or received_date = '20180128' or received_date = '20180129' or received_date = '20180130' or received_date = '20180131' or received_date = '20180201' or received_date = '20180202' or received

In [17]:
sdf.printSchema

<bound method DataFrame.printSchema of DataFrame[received_date: string, site_id: string, subscription_id: string, event_uuid: string, added_expiry_years: string, bank_code: string, bank_name: string, billing_country: string, bin: string, card_brand: string, card_category: string, card_class: string, card_usage: string, cc_expiration_date: string, cid: string, date_increment: string, day_of_month: int, day_of_week: string, exp_setting_id: string, funding_source: string, issuer_country: string, merchant_descriptor: string, merchant_number: string, mid_entity_code: string, new_status: string, order_entity_code: string, payment_amount: string, payment_amount_usd: string, payment_currency: string, payment_method_id: string, payment_service_id: string, renew_att_num: string, request_amount: string, request_amount_usd: string, request_currency: string, requisition_id: string, response_code: string, response_message: string, retry_rule_id: string, status: string, subsegment_id: string, time_of

In [54]:
# # sdf.select('renew_att_num','new_status','status').show(20)
sdf[(sdf.renew_att_num == '2') & (sdf.success == 1)].select('subscription_id', 'day_of_month', 'renew_att_num','new_status','success', 'received_date', 'site_id').show(5)
# # sdf[(sdf.success == 1)].select('renew_att_num','new_status','success', 'received_date').show(5)
# # sdf[(sdf.renew_att_num == '1')].select('renew_att_num','new_status','success', 'received_date').show(5)

# sdf.count()

+---------------+------------+-------------+----------+-------+-------------+-------+
|subscription_id|day_of_month|renew_att_num|new_status|success|received_date|site_id|
+---------------+------------+-------------+----------+-------+-------------+-------+
|    11004136101|           7|            2| Completed|      1|     20180307|adbehkr|
|    11146085001|           7|            2| Completed|      1|     20180307|adbehkr|
|     9341769801|           7|            2| Completed|      1|     20180307|adbehme|
|     3662874901|           7|            2| Completed|      1|     20180307|avastbr|
|    10232084101|           7|            2| Completed|      1|     20180307|samsung|
+---------------+------------+-------------+----------+-------+-------------+-------+
only showing top 5 rows



In [12]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import *
import datetime

df_decline_type = pd.read_csv(WORK_DIR + 'Decline_Type.csv')

msg_group = { 'declined' : 'decline', 
             'do_not_honor' : 'do not honor', 
             'txn_refused' : 'refuse', 
             'attempt_lower_amount' : 'lower amount',
            'Insufficient Funds' : 'insufficient',
            'not_allowed' : 'not allowed',
            'correct_cc_retry' : 'correct card',
            'invalid_cc' : 'invalid card',
            'lost_stolen' : 'lost or stolen',
            'invalid_account' : 'invalid account',
            'do_not_try' : 'do not try',
            'expired_card' : 'expired',
            'pickup_card' : 'pick',
            'blocked_first_used' : 'blocked',
            'invalid_txn' : 'invalid trans',
            'restricted_card' : 'restricted',
            'not_permitted' : 'not permitted',
            'expired card' : 'expired card',
            'unable to determine format' : 'determine format',
            'system error' : 'error'
            }

def group_response_msg(msg):
    other = 'Base'
    if isinstance(msg, str) == False:
        return other
    
    msg_lower = msg.lower()
    for key, val in msg_group.items():
        if val in msg_lower:
            return key
        
    return other   

def merge_by_sub(group):

    first = group[ group['renew_att_num'] == '1' ]
    
    if first.empty:
        group['first_attempt_date'] = ''
        group['first_day_of_month'] = 0
        group['first_transaction_hour'] = ''
        group['first_response_code'] = ''
        group['first_response_message'] = ''
        group['first_response_group'] = ''
        group['first_decline_type'] = ''
        group['days_between'] = 0
        return group
    
    last = group[ group['renew_att_num'] == group['renew_att_num'].max() ]
        
    first_attempt_date = first['transaction_date_in_string'].iloc[0]
    last_attempt_date = last['transaction_date_in_string'].iloc[0]
     
    
    last['first_attempt_date'] = first_attempt_date
    last['first_day_of_month'] = first['day_of_month'].iloc[0].astype(int)
    last['day_of_month'] = last['day_of_month'].astype(int)
    last['first_transaction_hour'] = first['transaction_hour'].iloc[0]
    last['first_response_code'] = first['response_code'].iloc[0]
    
    first_response_msg = first['response_message'].iloc[0]
    last['first_response_message'] = first_response_msg  
    last['first_response_group'] =   first['response_message'].apply(group_response_msg).iloc[0]
    
    last['first_decline_type'] = decline_type(first_response_msg)
    last['days_between'] =  days_between(first_attempt_date, last_attempt_date)
   

    return last

def decline_type(response_msg):
    '''Converts to decline_type based on the given response_msg'''
    dec_type = df_decline_type[df_decline_type['DECLINE_TEXT'] == response_msg]['DECLINE_TYPE']
    if dec_type.empty or dec_type.iloc[0] == 'Base' :
        return group_response_msg(response_msg)
    else:
        return dec_type.iloc[0]

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)

    return abs((d2 - d1).days)


# @pandas_udf(sdf.schema, PandasUDFType.GROUPED_MAP)
@pandas_udf(grouped_schema, PandasUDFType.GROUPED_MAP)
def merge_by_sub_spark(spark_df):
    print('Before apply ....... ')
#     result = pd.DataFrame(spark_df.groupby(['subscription_id', 'subsegment_id']).apply(merge_by_sub))
#     result = pd.DataFrame(spark_df).groupby(['subscription_id', 'subsegment_id']).apply(merge_by_sub)
#     result = spark_df.groupby(['subscription_id', 'subsegment_id']).apply(merge_by_sub)
#     result = pdf.groupby(['subscription_id', 'subsegment_id'], sort=False).apply(merge_by_sub)
    pdf = pd.DataFrame(spark_df)
    result = pdf.groupby(['subscription_id', 'subsegment_id']).apply(merge_by_sub)
#     if result.empty:
#         result = pdf
    print(result['site_id']) 
    return result

In [17]:
# pdf = sdf.filter(sdf.renew_att_num.isNotNull()).groupby("subscription_id", "subsegment_id").apply(merge_by_sub_spark).toPandas()
# pdf

pdf = sdf.filter(sdf.renew_att_num.isNotNull()).groupby("subscription_id", "subsegment_id").apply(merge_by_sub_spark)

In [16]:
pdf.select('subscription_id','renew_att_num','new_status','success', 'received_date', 'first_attempt_date', 'first_day_of_month', 'first_decline_type', 'days_between').show(10)

NameError: name 'pdf' is not defined

In [35]:
sdf.filter(sdf.renew_att_num.isNotNull()).groupby("subscription_id", "subsegment_id").apply(merge_by_sub_spark)\
    .filter('days_between > 0')\
    .show(5)

+-------------+-------+---------------+--------------------+------------------+----------------+--------------------+---------------+------+----------+--------------------+----------+--------------------+------------------+----------------+--------------+------------+-----------+--------------+--------------+--------------+-------------------+--------------------+-----------------+----------+-----------------+--------------+------------------+----------------+-----------------+------------------+-------------+--------------+------------------+----------------+--------------+-------------+--------------------+-------------+------+-------------+-------------------+-------------------+--------------------------+----------------+--------------+----------------+-------+-------------------+------------------+----------------------+-------------------+----------------------+--------------------+------------------+------------+
|received_date|site_id|subscription_id|          event_uuid|added_

In [13]:
# All sites
pdf_all = sdf.filter(sdf.renew_att_num.isNotNull()).groupby("subscription_id", "subsegment_id")\
    .apply(merge_by_sub_spark)\
    .filter('days_between > 0')\
    .toPandas()
pdf_all

,received_date,site_id,subscription_id,event_uuid,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,retry_rule_id,status,subsegment_id,time_of_day_rule_id,transaction_date,transaction_date_in_string,transaction_hour,transaction_id,transaction_type,success,first_attempt_date,first_day_of_month,first_transaction_hour,first_response_code,first_response_message,first_response_group,first_decline_type,days_between
0,20180418,avast,10001280501,673671cc-cc57-400c-b872-8ff9efb09a6d,None,NON3DS,Standard Chartered Bank Tanzania Limited,TZ,478680,Visa,Visa Gold,Consumer,NON USA Consumer Debit - No Pin Access,0322,None,None,18,WEDNESDAY,None,Debit,TZ,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,79.99,79.99,USD,Visa,netgiro-bms,3,79.99,79.99,USD,46971082700,27050,Do not try again/use alternate payment card.[04] [] [111800] [111800],None,New,13258980000,,2018-04-18 06:21:29,2018-04-18 01:21:29,01,6681738323,Authorize,0,2018-04-03 01:07:59,3,01,27050,Do not try again/use alternate payment card.[04] [] [111800] [111800],do_not_try,Hard Fraud,15
1,20180418,avastbr,10001441601,11cc8ab0-fe20-4e80-9d8c-2a7a856e7f17,None,NON3DS,ITAU UNIBANCO S.A.,BR,539083,MasterCard,(Unembossed) Standard MasterCard Card,Consumer,True credit (No PIN/Signature capability),1223,None,None,18,WEDNESDAY,None,Credit,BR,Avast Brazil,1883448942,DR_BRAZIL-ENTITY,Declined,DR_BRAZIL-ENTITY,99,29.018051,BRL,MasterCard,netgiro-br,3,99,29.018051,BRL,46969296500,27001,Transaction refused[62] [null] [] [],None,New,13259159000,,2018-04-18 06:02:11,2018-04-18 01:02:11,01,6681601083,Authorize,0,2018-04-08 01:03:36,8,01,27001,Transaction refused[62] [null] [] [],txn_refused,Hard Fraud,10
2,20180418,avast,10001635101,8bc95f07-77b3-4bd4-abfa-5fee6658eb63,None,57E041B0B2F3D9A5,Metabank,US,435880,Visa,Visa Classic,Consumer,Debit Hybrid (PIN and Signature),0625,None,None,18,WEDNESDAY,None,Prepaid,US,DRI*AVAST Software,311009012882,DR_INC-ENTITY,Declined,DR_INC-ENTITY,53.99,53.99,USD,Visa,firstdata,3,53.99,53.99,USD,46969386300,402,12: INV TRAN,None,New,13259490100,,2018-04-18 06:03:02,2018-04-18 01:03:02,01,6681607843,Authorize,0,2018-04-03 01:10:44,3,01,402,12: INV TRAN,Base,Base,15
3,20180425,defendbr,10001895901,9f5ecc0d-2529-4930-9fc5-93daf88c9e74,3,899EBF821D876833,ITAU UNIBANCO S.A.,BR,521397,MasterCard,Platinum MasterCard Card,Consumer,True credit (No PIN/Signature capability),0620,None,None,25,WEDNESDAY,None,Credit,BR,DRI*BitDefender,75FA81DE-5AC5-5DCF-E97C-8F80A08ADE1E,DR_BRAZIL-ENTITY,Declined,DR_BRAZIL-ENTITY,229.99,66.742732,BRL,MasterCard,pagador,3,229.99,66.742732,BRL,47202743400,530,7: Do Not Honor,None,New,13259627500,,2018-04-25 06:01:35,2018-04-25 01:01:35,01,6706709733,Authorize,0,2018-04-16 01:01:42,16,01,530,7: Do Not Honor,do_not_honor,do_not_honor,9
4,20180418,avast,10001966301,1bf185fb-88cd-457c-85a2-a697021fc538,None,0FDA420D7B7FDBDC,"Chase Bank USA, National Association",US,424631,Visa,Visa Business,Business,True credit (No PIN/Signature capability),0718,None,None,18,WEDNESDAY,None,Credit,US,DRI*AVAST Software,311009012882,DR_INC-ENTITY,Declined,DR_INC-ENTITY,64.94,64.94,USD,Visa,firstdata,3,64.94,64.94,USD,46969539100,201,14: INV ACCT NUM,None,New,13259788300,,2018-04-18 06:05:07,2018-04-18 01:05:07,01,6681621783,Authorize,0,2018-04-03 01:12:21,3,01,201,14: INV ACCT NUM,Base,Base,15
5,20180519,adbehru,10002261401,0d1fbb95-019b-4f5f-8c53-f96ae78c2d0a,None,NON3DS,SBERBANK of Russia,RU,427609,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,1018,RETRY_DECLINED.2,None,19,SATURDAY,None

In [20]:
pdf_all['received_date'].value_counts().sort_index()
# 
# pdf_all[pdf_all['subscription_id'] == '10001280501']
# pdf_all.size

20180102        3
20180103       85
20180104      618
20180105      690
20180106     1486
20180107     1522
20180108     2157
20180109     2098
20180110     2103
20180111     2710
20180112     4177
20180113     5104
20180114     4988
20180115     5322
20180116     9145
20180117    25668
20180118     9557
20180119     9994
20180120     9636
20180121    10874
20180122    10187
20180123    10405
20180124    10211
20180125    10610
20180126    10133
20180127    11197
20180128    10798
20180129     9552
20180130     9915
20180131     9336
            ...  
20180515    13186
20180516    12863
20180517    14349
20180518    13551
20180519    13474
20180520    13278
20180521    14189
20180522    13869
20180523    12490
20180524    12971
20180525    12899
20180526    13396
20180527    12245
20180528    14086
20180529    12239
20180530    13218
20180531    14062
20180601    13909
20180602    13480
20180603    13241
20180604    14019
20180605    12829
20180606    20153
20180607    19335
20180608  

In [21]:
# pdf_all.size

csv_name = str('pdf_all_site_%s_%s.csv' % (start_date, end_date))
csv_file_name = WORK_DIR + csv_name
pdf_all.to_csv(csv_file_name)

csv_file_name

'/drhdev/workspace/ml/pdf_all_site_2018-01-01_2018-06-30.csv'

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 30924)
Traceback (most recent call last):
  File "/drhdev/anaconda3/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/drhdev/anaconda3/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/drhdev/anaconda3/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/drhdev/anaconda3/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/drhdev/spark-2.3.0-bin-hadoop2.7/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/drhdev/spark-2.3.0-bin-hadoop2.7/python/pyspark/serializers.py", line 685, in read_int
    raise EOFError
EOFError
----------------------------------------


In [50]:
pdf_all['site_id'].value_counts()

avast       598381
avgstore    295098
kasperus    190715
adbehap     127795
avastbr     102684
adbehru      71398
adbehkr      59664
tmamer       57980
adbehbr      52632
adbehme      41675
adbeheu      36377
mfeus        28470
kasperde     15522
cppro        13453
kasperuk     12893
mfeeu        12539
tmapac       10958
kasperla      9845
kasperbr      9778
adbehil       8303
nvsubs        7023
kaspersk      6630
adbehtw       6508
tmemea        6384
kasper        6312
kaspergl      3851
defendde      3763
tmecon        3440
kasucpus      3187
adsk          2972
             ...  
adskkr         317
techsm         260
adskbr         230
defenduk       220
adbecnn        213
siplm1         206
defendwo       175
avastjp        170
defender       159
kasucpru       147
cybscrub       132
kasucpkr       108
nuancesm       101
kasucpla        82
ca              65
adskcct         55
tmpseu          47
tmoemap         47
tmpsus          40
tmpsap          26
kasperoe        25
cajapan     

In [41]:
pdf = sdf.filter(sdf.renew_att_num.isNotNull()).groupby("subscription_id", "subsegment_id")\
    .apply(merge_by_sub_spark)\
    .filter('days_between > 0')\
    .toPandas()
# sdf.filter(sdf.renew_att_num.isNotNull()).select('subscription_id', 'subsegment_id','renew_att_num','new_status','success', 'received_date').show(5)
# filter(sdf.col("renew_att_num").isNotNull())
# sdf.filter(sdf.renew_att_num.isNotNull()).groupby("subscription_id", "subsegment_id").apply(merge_by_sub_spark).show()
pdf

,received_date,site_id,subscription_id,event_uuid,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,retry_rule_id,status,subsegment_id,time_of_day_rule_id,transaction_date,transaction_date_in_string,transaction_hour,transaction_id,transaction_type,success,first_attempt_date,first_day_of_month,first_transaction_hour,first_response_code,first_response_message,first_response_group,first_decline_type,days_between
0,20180418,avast,10001280501,673671cc-cc57-400c-b872-8ff9efb09a6d,None,NON3DS,Standard Chartered Bank Tanzania Limited,TZ,478680,Visa,Visa Gold,Consumer,NON USA Consumer Debit - No Pin Access,0322,None,None,,WEDNESDAY,None,Debit,TZ,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,79.99,79.99,USD,Visa,netgiro-bms,3,79.99,79.99,USD,46971082700,27050,Do not try again/use alternate payment card.[04] [] [111800] [111800],None,New,13258980000,,2018-04-18 06:21:29,2018-04-18 01:21:29,01,6681738323,Authorize,0,2018-04-03 01:07:59,3,01,27050,Do not try again/use alternate payment card.[04] [] [111800] [111800],do_not_try,Hard Fraud,15
1,20180418,avast,10001635101,8bc95f07-77b3-4bd4-abfa-5fee6658eb63,None,57E041B0B2F3D9A5,Metabank,US,435880,Visa,Visa Classic,Consumer,Debit Hybrid (PIN and Signature),0625,None,None,,WEDNESDAY,None,Prepaid,US,DRI*AVAST Software,311009012882,DR_INC-ENTITY,Declined,DR_INC-ENTITY,53.99,53.99,USD,Visa,firstdata,3,53.99,53.99,USD,46969386300,402,12: INV TRAN,None,New,13259490100,,2018-04-18 06:03:02,2018-04-18 01:03:02,01,6681607843,Authorize,0,2018-04-03 01:10:44,3,01,402,12: INV TRAN,Base,Base,15
2,20180418,avast,10001966301,1bf185fb-88cd-457c-85a2-a697021fc538,None,0FDA420D7B7FDBDC,"Chase Bank USA, National Association",US,424631,Visa,Visa Business,Business,True credit (No PIN/Signature capability),0718,None,None,,WEDNESDAY,None,Credit,US,DRI*AVAST Software,311009012882,DR_INC-ENTITY,Declined,DR_INC-ENTITY,64.94,64.94,USD,Visa,firstdata,3,64.94,64.94,USD,46969539100,201,14: INV ACCT NUM,None,New,13259788300,,2018-04-18 06:05:07,2018-04-18 01:05:07,01,6681621783,Authorize,0,2018-04-03 01:12:21,3,01,201,14: INV ACCT NUM,Base,Base,15
3,20180419,avast,10002779101,1528c8cc-89ae-405a-a657-dece1eeaf8c9,None,NON3DS,"Citibank, National Association",AE,492082,Visa,Visa Platinum,Consumer,True credit (No PIN/Signature capability),0618,None,None,,THURSDAY,None,Credit,AE,DRI*AVAST Software,1156962062,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,208.95,56.89589,AED,Visa,netgiro-seb,3,208.95,56.89589,AED,47004294700,27001,Transaction refused[05] [000000] [] [],None,New,13263437500,,2018-04-19 06:08:57,2018-04-19 01:08:57,01,6685403713,Authorize,0,2018-04-04 01:09:51,4,01,27001,Transaction refused[05] [000000] [] [],txn_refused,txn_refused,15
4,20180419,avast,10002829801,9f80ce79-91ff-4f8a-9971-b87bf84da4e8,5,RB,La Banque Postale,RE,497040,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0123,None,5,,THURSDAY,735801,Debit,FR,DRI*AVAST Software,1785390597,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,49.99,61.927612,EUR,Visa,netgiro-bnp,3,49.99,61.927612,EUR,47004126200,27001,Transaction refused[56] [] [] [],None,New,13263542400,,2018-04-19 06:06:58,2018-04-19 01:06:58,01,6685387293,Authorize,0,2018-04-04 01:10:36,4,01,27001,Transaction refused[56] [] [] [],txn_refused,txn_refused,15
5,20180419,avast,10002829801,f7436e4d-5994-4938-9a46-0ef862fef567,5,NON3DS,La Banque Postale,RE,497040,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0123,RETRY_DECLINED.2,5,,THURSDAY,735801,Debit,FR,DRI*AVAST Software,1411163460,DR_IRELAN

In [47]:
pdf['received_date'].value_counts()

20180609    7145
20180610    7034
20180612    6562
20180611    6561
20180608    6512
20180317    4663
20180316    4405
20180320    4395
20180321    4297
20180318    4273
20180319    4253
20180605    4212
20180324    4182
20180515    4155
20180328    4109
20180326    4099
20180323    4049
20180517    4048
20180325    4045
20180329    4001
20180530    3937
20180330    3932
20180531    3920
20180510    3914
20180518    3900
20180331    3863
20180603    3840
20180322    3835
20180414    3832
20180509    3821
            ... 
20180420    3392
20180410    3392
20180421    3388
20180523    3381
20180502    3381
20180521    3378
20180417    3369
20180416    3360
20180527    3358
20180528    3343
20180604    3335
20180408    3332
20180419    3330
20180508    3315
20180418    3285
20180424    3256
20180507    3248
20180625    1643
20180311     708
20180313     665
20180314     615
20180312     600
20180315     575
20180306     364
20180308     352
20180307     347
20180309     343
20180310     3

In [32]:
pdf[pdf['success'] == '1']

,received_date,site_id,subscription_id,event_uuid,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,retry_rule_id,status,subsegment_id,time_of_day_rule_id,transaction_date,transaction_date_in_string,transaction_hour,transaction_id,transaction_type,success,first_attempt_date,first_day_of_month,first_transaction_hour,first_response_code,first_response_message,first_response_group,first_decline_type,days_between
329,20180101,avast,3120597009,f677bf29-4476-469d-96f7-3e5fad06184a,None,A1345846C88AABBD,ING Bank (Australia) Limited,AU,462263,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,11/17,None,None,01,Monday,None,Debit,AU,DRI*AVAST Software,I7A0010,DR_IRELAND-ENTITY,Completed,,59.99,46.88258,AUD,Visa,nab,2,59.99,46.88258,AUD,43972475200,100,08: Approved,None,New,12620850800,,2018-01-01 06:32:49,2018-01-01 00:32:49,00,6332700013,Authorize,1,,,,,,,,0
344,20180109,avast,3127182309,b554e9b9-964f-4e74-b1b5-872d483e9a62,None,D650921B2A1F9EE6,JPMorgan Chase Bank N.A. - Debit,US,478200,Visa,Visa Classic,Consumer,Debit Hybrid (PIN and Signature),0219,None,None,09,Tuesday,None,Debit,US,DRI*AVAST Software,311009012882,DR_INC-ENTITY,Completed,DR_INC-ENTITY,53.11,53.11,USD,Visa,firstdata,3,53.11,53.11,USD,44171616200,100,00: Approved,None,New,12607602200,,2018-01-09 06:22:54,2018-01-09 00:22:54,00,6355889623,Authorize,1,,,,,,,,0
355,20180112,avast,3132431709,5827e850-761f-4f1c-97ac-e5b8fa63dfa4,None,NON3DS,DEUTSCHER SPARKASSEN UND GIROVERBAND,DE,523254,MasterCard,MasterCard Credit Card (mixed BIN),Consumer,True credit (No PIN/Signature capability),0221,None,None,,Friday,None,Credit,DE,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,59.99,72.089983,EUR,MasterCard,netgiro-bms,2,59.99,72.089983,EUR,44249347000,0,SUCCESS[00] [420688] [111800] [111800],None,New,12623187500,,2018-01-12 06:25:40,2018-01-12 00:25:40,00,6364733193,Authorize,1,2018-01-02 00:32:37,02,00,27001,Transaction refused[null] [null] [null] [null],txn_refused,txn_refused,10
361,20180112,avast,3137520609,430f714e-0481-419f-b631-4d8ff9206b30,None,EB62896607F5E12D,NRL Federal Credit Union,US,490268,Visa,Visa Traditional Rewards,Consumer,True credit (No PIN/Signature capability),0319,None,None,,Friday,None,Credit,US,DRI*AVAST Software,311009012882,DR_INC-ENTITY,Completed,DR_INC-ENTITY,79.99,79.99,USD,Visa,firstdata,2,79.99,79.99,USD,44249165900,100,00: Approved,None,New,12668683800,,2018-01-12 06:23:28,2018-01-12 00:23:28,00,6364715923,Authorize,1,2018-01-02 00:22:11,02,00,530,51: DECLINED,declined,declined,10
379,20180117,avast,3146195809,e078693c-0750-43c7-a003-9e0bea255b21,None,NON3DS,"TARJETAS BANAMEX, SA DE CV, SOFOM E.R.",MX,518853,MasterCard,(Unembossed) Standard MasterCard Card,Consumer,True credit (No PIN/Signature capability),0119,None,None,,Wednesday,None,Credit,MX,DRI*AVAST Software,1013247426,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,999,53.216065,MXN,MasterCard,drwp-dlocal,3,999,53.216065,MXN,44382528800,100,SUCCESS[null] [null] [null] [null],None,New,12654694300,,2018-01-17 06:28:08,2018-01-17 00:28:08,00,6379516303,Authorize,1,2018-01-02 00:32:33,02,00,27001,Transaction refused[05] [000000] [] [],txn_refused,txn_refused,15
382,20180114,avast,3150082809,91657ab4-7d43-45a4-9830-a22fdf2ef4d6,None,FBECDFB8DF0E8CB6,"CHASE BANK USA, N.A.",US,552475,MasterCard,Platinum MasterCard Card,Consumer,True credit (No PIN/Signature capability),1218,None,None,,Sunday,None,Credit,US,DRI*AVAST Software,7236114,DR_INC-ENTITY,Completed,DR_INC-ENTITY,104.99,104.99,USD,MasterCard,litle,2,104.99,104.99